In [10]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("dane1.csv")
df.head()

In [ ]:
day_map = {
    "Mon": "poniedzialek",
    "Tue": "wtorek",
    "Wed": "sroda",
    "Thu": "czwartek",          ### Tlumaczenie skrotow
    "Fri": "piatek",
    "Sat": "sobota",
    "Sun": "niedziela",
}

df["created_at_day_pl"] = df["created_at"].astype(str).str.slice(0,3).map(day_map)  ### wyciagniecie skrotu dnia, podmiana skrotu na polska nazwe, zapis do nowej kolumny

df[["created_at", "created_at_day_pl"]].head(10)    ### Kontrolne wyswietlenie pierwszych 10


In [ ]:
month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",         ### Stworzenie mapy miesiecy
    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

df["user_created_at"].astype(str).str.slice(4,7)                ### Wyciagniecie skrotu miesiaca

df["user_created_at_month_num"] = df["user_created_at"].astype(str).str.slice(4,7).map(month_map)       ### Zmapowanie skrotu miesiaca na numer i zapisanie do nowej kolumny

df[["user_created_at", "user_created_at_month_num"]].head(10)         ### Kontrola czy dziala


In [ ]:
### Sprawdzenie czy dzialaja kolumny ktore zostaly zmienione
df[[
    "created_at",
    "created_at_day_pl",
    "user_created_at",
    "user_created_at_month_num"
]].head(10)

In [42]:
import re

tweet_link_pattern = re.compile (
    r"https?://twiter\.com/[^/]+/status/\d+(?:/photo/\d+)?"     ### Wrzozec linku do tweeta
)

### Pusty zbior na linki
tweet_links = set()

### Wyciagniecie linkow z kolumy *tweet_url
if "tweet_url" in df.columns:
  for x in df["tweet_url"].dropna().astype(str):
    match = tweet_link_pattern.search(x)
    if match:
      tweet_links.add(match.group(0))

### Wyciagniecie linkow z kolumny *media
if "media" in df.columns:
  for x in df["media"].dropna().astype(str):
    match = tweet_link_pattern.search(x)
    if match:
      tweet_links.add(match.group(0))

tweet_links = sorted(tweet_links)                   ### Zmiana zbioru na liste

len(tweet_links), tweet_links[:10]                   ### Sprawdzenie

(0, [])

In [ ]:
import re

url_pattern = re.compile(r"https?://\S+")     ### Regex do wycagania linkow

all_links = set()       ### Pusty zbior na linki

### Linki z kolumny *text
for t in df["text"].dropna().astype(str):
  for link in url_pattern.findall(t):
    all_links.add(link.rstrip(").,!?\"'")) #dodanie do zbioru i usuniecie podanych znakow

### Linki z kolumny *urls
if "urls" in df.columns:
  for x in df["urls"].dropna().astype(str):
    for link in url_pattern.findall(x):
      all_links.add(link.rstrip(").,!?\"'"))

all_links = sorted(all_links)           ### Zmiana zbioru na liste

len(all_links), all_links[:10]




In [ ]:
image_links = []

if"media" in df.columns:
  image_links = (
      df["media"]
      .dropna()                                                    # usuwanie pustych komorek
      .astype(str)                                                  # pewnosc operwoania na stringach
      .loc[lambda s: s.str.contains("/photo/", regex = False)]       # zostaja rekordy z linkami do zdjec
      .unique()                                                       # usuniecie duplikatow
      .tolist()                                                        # zmiana wyniku na liste Pyth
  )

image_links = sorted(image_links)   # sortowanie listy

len(image_links), image_links[:10]


In [92]:
import requests
import re

stopwords_url = "https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt"             ### Pobranie listy stopwords
response = set(requests.get(stopwords_url, timeout = 30).text.splitlines())

len(stopwords)    # kontrola ile mamy stopwords


### Funkcja czyszczaca tekst i usuwajaca stopwrods

def remove_stopwords_pl(text, stopwords_set):
    if not isinstance(text, str):
        return ""

    text = text.lower()

    # zostawiamy litery (rowniez polskie), cyfry i spacje
    text = re.sub(r"[^0-9a-ząćęłńóśźż\s]+", " ", text)

    tokens = text.split()
    tokens = [tok for tok in tokens if tok not in stopwords_set]

    return " ".join(tokens)

# stworzenie nowej kolumny z tekstem bez stopwords
df["text_without_stopwords"] = df["text"].apply(
    lambda x: remove_stopwords_pl(x, stopwords)
)

# podglad efektu
df[["text", "text_without_stopwords"]].head(5)


,text,text_without_stopwords
0,@beata_skwarska Warszawa 😀 https://t.co/W7BcyS...,beata skwarska warszawa https t w7bcysyund
1,Nieznani sprawcy podpalili kapliczkę nadrzewną...,nieznani sprawcy podpalili kapliczkę nadrzewną...
2,⚠️ Utrudnienia w komunikacji: L20 https://t.co...,utrudnienia komunikacji l20 https t dxakbokzf5
3,@LukaszKohut @moanrosa @LincaAgata @jan_jozef_...,lukaszkohut moanrosa lincaagata jan jozef lubo...
4,"Dzieci to największy skarb, o który musimy dba...",dzieci największy skarb musimy dbać wspieramyr...


In [95]:
df.to_csv("dane1_part1_clean.csv", index=False)

import json

with open("tweet_links.json", "w", encoding="utf-8") as f:
    json.dump(tweet_links, f, ensure_ascii=False, indent=2)

with open("all_links.json", "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)

with open("image_links.json", "w", encoding="utf-8") as f:
    json.dump(image_links, f, ensure_ascii=False, indent=2)


from google.colab import files
files.download("dane1_part1_clean.csv")
files.download("tweet_links.json")
files.download("all_links.json")
files.download("image_links.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>